In [12]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from ols import leastsq
%matplotlib inline

In [13]:
df = pd.read_csv('../data/lsigma_new.csv')
df.head()

,name,lum,sig,oh,ewhb,ion,te,ne,chb,z,ref,type,class,sigobs,photobs,out
0,UM238,40.024,1.270,7.891,1.554,0.520,4.186,2.938,0.233,0.01427,1,Gaussian Profile,G,FEROS,B&C,0
1,mrk557,40.668,1.761,8.697,0.996,-0.715,4.146,2.573,0.383,0.01328,1,Irregular Profile,I,COUDÉ,B&C,0
2,UM304,41.546,1.893,0.000,0.000,0.000,4.146,2.309,0.000,0.01570,14,Profile with Components,C,COUDÉ,Others,0
3,cts1001,40.810,1.683,7.961,1.775,0.059,4.173,2.927,0.189,0.02263,1,Irregular Profile,I,FEROS,B&C,0
4,UM306,40.245,1.282,8.184,1.375,0.344,4.065,1.423,0.082,0.01649,1,Gaussian Profile,G,FEROS,B&C,0


In [14]:
df81 = df[(df['sigobs']=="FEROS") & (df['photobs'] != "Others")]
df81.shape

(81, 16)

In [15]:
X = df81['sig'].to_numpy().reshape(-1,1)
y = df81['lum'].to_numpy()

Same results as that in **Table 6** in Bordalo & Telles (2011). 

# OLS

In [11]:
a, b, c, d, e = leastsq(X,y, method=1)
print('log_ L = {} +/- {} + {} +/- {} log_s; rms = {}'.format(a.round(2),b.round(3),c.round(2),d.round(3),e.round(2)))
a, b, c, d, e = leastsq(X,y, method=2)
print('log_ L = {} +/- {} + {} +/- {} log_s; rms = {}'.format(a.round(2),b.round(3),c.round(2),d.round(3),e.round(2)))
a, b, c, d, e = leastsq(X,y, method=3)
print('log_ L = {} +/- {} + {} +/- {} log_s; rms = {}'.format(a.round(2),b.round(3),c.round(2),d.round(3),e.round(2)))

log_ L = 36.21 +/- 0.323 + 3.01 +/- 0.229 log_s; rms = 0.37
log_ L = 34.52 +/- 0.377 + 4.18 +/- 0.269 log_s; rms = 0.44
log_ L = 35.49 +/- 0.322 + 3.51 +/- 0.23 log_s; rms = 0.39


# OLS my class

In [ ]:
class OLS(object):
    """
    Class to estimate the ordinary least-squares regression coefficients for
    OLS(X|Y), OLS(Y|X) and OLS bisector as decribed in Isobe et al. (1990).
    
    TO DO: slope and intercept variances
    
    References:
    Isobe, T., Feigelson, E. D., Akritas, M. G., & Babu, G. J. 1990, ApJ, 364, 104
    """
    def __init__(self, X, y, method=1):
        """
        X : numpy array (n, 1)
        y : numpy array (n, ) 
        """ 
        
        N = X.shape[0]
        K = 2 # number of regression parameters (slope and intercept, OLS)
        x_mean = X[:,0].mean()
        x_res = X[:,0] - x_mean
        y_mean = y.mean()
        y_res = y - y_mean
        Sxx = sum(x_res**2)
        Syy = sum(y_res**2)
        Sxy = sum(x_res*y_res)

        # OLS(Y|X)
        reg_direct = LinearRegression().fit(X, y)
        self.a1 = reg_direct.intercept_
        self.b1 = reg_direct.coef_[0]
        self.R2 = reg_direct.score(X, y)
        self.rms1 = np.sqrt(sum((y-reg_direct.predict(X))**2)/(N-K)) # in units of Y
        self.var_b1 = np.sqrt((1/Sxx**2)*(sum((x_res**2)*(y-self.b1*X[:,0]-y_mean+self.b1*x_mean)**2)))


        # OLS(X|Y)
        XX = X.reshape(-1)
        yy = y.reshape(-1,1)
        reg_inverse = LinearRegression().fit(yy, XX)
        self.a2 = -reg_inverse.intercept_/reg_inverse.coef_[0]
        self.b2 = 1/reg_inverse.coef_[0]
        y_hat = self.a2 + self.b2*XX
        self.rms2 = np.sqrt(sum((yy[:,0]-y_hat)**2)/(N-K)) # in units of Y
        self.var_b2 = np.sqrt((1/Sxy**2)*(sum((y_res**2)*(y-self.b2*X[:,0]-y_mean+self.b2*x_mean)**2)))
    
        covb12 = np.sum(x_res*y_res*(y_res-self.b2 * x_res)*(y_res-self.b1 * x_res))/(self.b1*Sxx**2)
        
        b1_1 = 1 + (self.b1**2)
        b2_1 = 1 + (self.b2**2)
        b1_b2 = (self.b1 + self.b2)**2
        
        # OLS bisector
        self.b3 = 1/(self.b1+self.b2) * (self.b1*self.b2 - 1 + np.sqrt((1+self.b1**2) * (1+self.b2**2)))
        self.a3 = y.mean() - self.b3*X.mean()
        y_hat = self.a3 + self.b3*X[:,0]
        self.rms3 = np.sqrt(sum((y-y_hat)**2)/(N-K)) # in units of Y 
        
        prefac = self.b3**2 / ( b1_b2 * b1_1 * b2_1)
        var = b2_1**2 * self.var_b1 + 2 * b1_1 * b2_1 * covb12 + b1_1**2 * self.var_b2
        self.var_b3 = np.sqrt(prefac*var)


In [ ]:
res = OLS(X,y)
print('log_ L = {} +/- {} + {} +/- {} log_s  (rms = {})'.format(res.a1.round(2),'',res.b1.round(2), res.var_b1.round(3),res.rms1.round(2)))
print('log_ L = {} +/- {} + {} +/- {} log_s  (rms = {})'.format(res.a2.round(2),'',res.b2.round(2), res.var_b2.round(3),res.rms2.round(2)))
print('log_ L = {} +/- {} + {} +/- {} log_s  (rms = {})'.format(res.a3.round(2),'',res.b3.round(2), res.var_b3.round(3),res.rms3.round(2)))

Using ```OLS``` from ```statsmodels```:

In [ ]:
X = sm.add_constant(X)
results = sm.OLS(y,X).fit()
print(results.summary())

In [ ]:
# With linear algebra 
# numpy arrays

In [ ]:
N = X.shape[0]
p = X.shape[1] + 1  # plus one because LinearRegression adds an intercept term

X_with_intercept = np.empty(shape=(N, p), dtype=float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = X

In [ ]:
beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y
print(beta_hat)

In [ ]:
y_hat = beta_hat[0] + beta_hat[1]*X
residuals = y - y_hat
residual_sum_of_squares = residuals.T @ residuals

In [ ]:
sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

??

In [ ]:
X = pd.DataFrame(np.random.randn(1000,3), columns=['X1','X2','X3'])
y = pd.DataFrame(np.random.randn(1000,1), columns=['Y'])        

model = LinearRegression()
model.fit(X=X, y=y)

In [ ]:
for i in range(X.shape[1]):
    plt.scatter(X.iloc[:,i],y)
plt.show()

In [ ]:
N = len(X)
p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

In [ ]:
X_with_intercept = np.empty(shape=(N, p), dtype=float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = X.values

In [ ]:
beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y.values
print(beta_hat)

In [ ]:
y_hat = model.predict(X)
residuals = y.values - y_hat
residual_sum_of_squares = residuals.T @ residuals
sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")